In [1]:
from confluent_kafka import Consumer, Producer, KafkaError, KafkaException
import sys




In [57]:
from kafka import TopicPartition, KafkaConsumer

# Create the consumer
consumer = KafkaConsumer(bootstrap_servers='localhost:9092',
                         auto_offset_reset='earliest',  # Start consuming from the earliest available message
                         enable_auto_commit=False)       # Disable auto-commit of offsets

# Manually assign a partition
topic_partition = TopicPartition('tts_messages', 0)  # Assign partition 0
consumer.assign([topic_partition])

messages = [0] * 100000
for msg in consumer:
    messages[msg.offset] = msg.value
    msg.timestamp
    if msg.offset == 99999:
        break


100000

In [ ]:
# from confluent_kafka import Consumer

# conf = {'bootstrap.servers': 'host1:9092,host2:9092',
#         'group.id': 'foo',
#         'enable.auto.commit': 'false',
#         'auto.offset.reset': 'earliest'}
# # 
# consumer = Consumer(conf)

In [ ]:
# running = True

# def basic_consume_loop(consumer, topics):
#     try:
#         consumer.subscribe(topics)

#         while running:
#             msg = consumer.poll(timeout=1.0)
#             if msg is None: continue

#             if msg.error():
#                 if msg.error().code() == KafkaError._PARTITION_EOF:
#                     # End of partition event
#                     sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
#                                      (msg.topic(), msg.partition(), msg.offset()))
#                 elif msg.error():
#                     raise KafkaException(msg.error())
#             else:
#                 msg_process(msg)
#     finally:
#         # Close down consumer to commit final offsets.
#         consumer.close()

# def shutdown():
#     running = False

In [38]:
from kafka import KafkaProducer
# config = {
#     "bootstrap_servers": "localhost:9092",
#     # 'client.id': socket.gethostname()
# }

producer = KafkaProducer(bootstrap_servers='localhost:9092')
print(producer.bootstrap_connected())


True


In [36]:
producer.flush()


In [79]:
import json
num = 1000
objects = [{
        "message":"Today I was a given a task to build a system that can process audio files and convert them to text. I have a lot of experience in this area, having worked on similar projects in the past. I am confident that I can handle this task and deliver a high-quality solution.", 
        "language":"en", 
        "voice":"female",
        "speed":1.0,
        "shm_name":f"shm_{i}",
        "id":i
    } for i in range(num)]

objects_json = [json.dumps(obj) for obj in objects]
objects_bytes = [obj.encode('utf-8') for obj in objects_json]

In [82]:

for i in range(num):
    producer.send('tts_messages', value=objects_bytes[i])


In [70]:
from kafka.admin import KafkaAdminClient, NewTopic
def create_topic_with_retention(
        topic_name: str,
        num_partitions: int = 1,
        replication_factor: int = 1,
        retention_ms: int = 604800000,
    ):
    admin_client = KafkaAdminClient(
        bootstrap_servers='localhost:9092'
    )
    topic_configs = {"retention.ms": str(retention_ms)}
    new_topic = NewTopic(
        name=topic_name,
        num_partitions=num_partitions,
        replication_factor=replication_factor,
        topic_configs=topic_configs,
    )
    try:
        admin_client.create_topics([new_topic])
        print(
            f"Topic '{topic_name}' created successfully with retention period of {retention_ms} ms."
        )
    except Exception as e:
        print(f"Failed to create topic: {e}")
    finally:
        admin_client.close()
create_topic_with_retention("tts_messages", retention_ms=1000 * 60 * 60) # 1 hour

Topic 'tts_messages' created successfully with retention period of 3600000 ms.
